## 1️⃣ Setup and Installation

Install required packages:

In [ ]:
# Install Microsoft Agent Framework and dependencies
!pip install agent-framework>=1.0.0b251223 python-dotenv openai>=1.55.0 -q

print("✅ Packages installed successfully!")

## 2️⃣ Configure GitHub Models Authentication

Set up your GitHub token and model configuration:

In [ ]:
import os
import getpass
from dotenv import load_dotenv

# Try to load from .env file first
load_dotenv()

# Get GitHub token (will prompt if not in environment)
if not os.getenv("GITHUB_TOKEN"):
    github_token = getpass.getpass("Enter your GitHub Personal Access Token: ")
    os.environ["GITHUB_TOKEN"] = github_token
else:
    github_token = os.getenv("GITHUB_TOKEN")

# Configuration
GITHUB_MODEL = os.getenv("GITHUB_MODEL", "gpt-4o-mini")
GITHUB_BASE_URL = os.getenv("GITHUB_BASE_URL", "https://models.inference.ai.azure.com")

print("✅ Authentication configured")
print(f"   Model: {GITHUB_MODEL}")
print(f"   Endpoint: {GITHUB_BASE_URL}")
print(f"   Token: {github_token[:20]}...")

## 3️⃣ Initialize Microsoft Agent Framework

Create an OpenAI-compatible client pointing to GitHub Models:

In [ ]:
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

# Create client for GitHub Models
github_client = OpenAIChatClient(
    model_id=GITHUB_MODEL,
    api_key=github_token,
    base_url=GITHUB_BASE_URL
)

print("✅ MAF client initialized")
print(f"   Client type: {type(github_client).__name__}")
print(f"   Model: {GITHUB_MODEL}")

## 4️⃣ Create a Basic Agent

Create a simple AI agent using GitHub Models:

In [ ]:
# Create agent with GitHub Models
agent = ChatAgent(
    chat_client=github_client,
    instructions=(
        "You are a helpful AI assistant powered by GitHub Models. "
        "You provide clear, concise answers and acknowledge that "
        "you're using GitHub Models for inference."
    ),
    name="GitHubModelsAgent"
)

print("✅ Agent created successfully")
print(f"   Name: {agent.name}")
print(f"   Instructions: {agent.instructions[:80]}...")

## 5️⃣ Test Basic Agent Interaction

Run a simple query:

In [ ]:
import asyncio

# Define async function for agent interaction
async def test_basic_agent():
    query = "What are the key benefits of using AI agents in software development?"
    print(f"🤔 Query: {query}\n")
    
    result = await agent.run(query)
    print(f"🤖 Response:\n{result}")
    return result

# Run the async function
response = await test_basic_agent()

## 6️⃣ Add Function Tools

Enhance the agent with custom function tools:

In [ ]:
from typing import Annotated

# Define custom tools
def get_weather(location: Annotated[str, "City and state, e.g. Seattle, WA"]) -> str:
    """Get current weather for a location."""
    # Simulated weather data
    weather_db = {
        "Seattle, WA": "🌧️ Rainy, 52°F",
        "San Francisco, CA": "☁️ Partly cloudy, 65°F",
        "New York, NY": "❄️ Cold, 32°F",
    }
    return weather_db.get(location, f"No data for {location}")

def calculate_cost(
    hours: Annotated[float, "Hours worked"],
    rate: Annotated[float, "Hourly rate in dollars"]
) -> str:
    """Calculate total cost."""
    total = hours * rate
    return f"${total:.2f} ({hours} hours × ${rate}/hour)"

# Create agent with tools
tool_agent = ChatAgent(
    chat_client=github_client,
    instructions="You are a helpful assistant with access to tools. Use them when appropriate.",
    tools=[get_weather, calculate_cost],
    name="ToolEnabledAgent"
)

print("✅ Agent with tools created")
print(f"   Tools available: {len(tool_agent.tools)}")

## 7️⃣ Test Agent with Tools

Query the agent to trigger tool usage:

In [ ]:
async def test_tools():
    queries = [
        "What's the weather in Seattle, WA?",
        "If I work 40 hours at $75/hour, what's my total pay?"
    ]
    
    for query in queries:
        print(f"\n🤔 Query: {query}")
        result = await tool_agent.run(query)
        print(f"🤖 Response: {result}")
        
        # Rate limiting
        await asyncio.sleep(3)

await test_tools()

## 8️⃣ Multi-Agent Workflow

Create a workflow with multiple specialized agents:

In [ ]:
# Create specialized agents
research_agent = ChatAgent(
    chat_client=OpenAIChatClient(
        model_id=GITHUB_MODEL,
        api_key=github_token,
        base_url=GITHUB_BASE_URL
    ),
    instructions="You are a research specialist. Gather key facts and information.",
    name="Researcher"
)

writer_agent = ChatAgent(
    chat_client=OpenAIChatClient(
        model_id=GITHUB_MODEL,
        api_key=github_token,
        base_url=GITHUB_BASE_URL
    ),
    instructions="You are a technical writer. Create clear, well-structured content.",
    name="Writer"
)

print("✅ Multi-agent workflow created")
print(f"   Agent 1: {research_agent.name}")
print(f"   Agent 2: {writer_agent.name}")

## 9️⃣ Execute Multi-Agent Workflow

Sequential agent execution:

In [ ]:
async def run_workflow():
    topic = "Benefits of using AI agents in enterprise software"
    print(f"📋 Topic: {topic}\n")
    
    # Step 1: Research
    print("📚 Step 1: Research...")
    research = await research_agent.run(
        f"Research key benefits of: {topic}"
    )
    print(f"✅ Research complete ({len(research)} chars)\n")
    await asyncio.sleep(5)
    
    # Step 2: Write
    print("✍️  Step 2: Writing...")
    article = await writer_agent.run(
        f"Write a concise article based on this research:\n{research}"
    )
    print(f"✅ Article complete ({len(article)} chars)\n")
    
    print("=" * 70)
    print("📄 Final Article:")
    print("=" * 70)
    print(article)

await run_workflow()

## 🔟 Comparison: GitHub Models vs Azure Options

### Configuration Comparison

Let's compare the code patterns:

In [ ]:
# Show configuration comparison
print("=" * 70)
print("Configuration Comparison")
print("=" * 70)

print("\n1️⃣ GitHub Models (Current Setup):")
print("""
from agent_framework.openai import OpenAIChatClient

client = OpenAIChatClient(
    model_id="gpt-4o-mini",
    api_key=os.getenv("GITHUB_TOKEN"),
    base_url="https://models.inference.ai.azure.com"
)

agent = ChatAgent(chat_client=client, instructions="...")
""")

print("\n2️⃣ Azure OpenAI:")
print("""
from agent_framework.azure import AzureOpenAIChatClient

client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name="gpt-4o-mini",
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

agent = ChatAgent(chat_client=client, instructions="...")
""")

print("\n3️⃣ Azure AI Foundry V2 (Server-Side):")
print("""
from agent_framework.azure import AzureAIClient
from azure.identity import DefaultAzureCredential

client = AzureAIClient(
    endpoint=os.getenv("AZURE_AI_PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential()
)

agent = client.create_agent(
    model=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME"),
    instructions="...",
    name="MyAgent",
    use_latest_version=True  # Server-side persistence
)
""")

### Feature Comparison Table

| Feature | GitHub Models | Azure OpenAI | Foundry V2 |
|---------|---------------|--------------|------------|
| **Cost** | Free (rate-limited) | Pay-per-token | Service + tokens |
| **Setup** | < 5 min | 15-30 min | 30-60 min |
| **Rate Limits** | ~15 RPM | 60-1000+ RPM | 60-1000+ RPM |
| **Client-Side Agents** | ✅ | ✅ | ✅ |
| **Server-Side Agents** | ❌ | ❌ | ✅ |
| **Production Ready** | ❌ | ✅ | ✅ |
| **Best For** | Development | Production | Enterprise |

### When to Use Each

**GitHub Models:**
- ✅ Learning and experimentation
- ✅ Rapid prototyping
- ✅ Budget-conscious projects
- ✅ Testing different models (Llama, Phi)

**Azure OpenAI:**
- ✅ Production applications
- ✅ Higher rate limits needed
- ✅ Enterprise security
- ✅ SLA requirements

**Azure AI Foundry V2:**
- ✅ Multi-agent orchestration
- ✅ Server-side persistence
- ✅ Agent versioning
- ✅ Full governance

---

## 🎯 Key Takeaways

1. **Same MAF Code**: Agent code is largely identical across providers
2. **Easy Migration**: Only client configuration changes
3. **Development Path**: Start with GitHub Models → Move to Azure for production
4. **Foundry V2 Benefits**: Server-side agents only available with Azure AI Foundry

---

## 📚 Next Steps

1. **Explore Models**: Try different GitHub Models (Llama, Phi, Mistral)
2. **Add Custom Tools**: Create your own function tools
3. **Complex Workflows**: Implement parallel agent execution
4. **Migrate to Azure**: When ready for production

---

## 🔗 Resources

- **GitHub Models**: https://github.com/marketplace/models
- **MAF Documentation**: https://aka.ms/agent-framework
- **MAF Repository**: https://github.com/microsoft/agent-framework
- **Azure AI Foundry**: https://ai.azure.com

---

**Happy Agent Building! 🚀**